# exploração dos dados

In [5]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [7]:
df = pd.read_csv("../raw_data/SP/INMET_SE_SP_A705_BAURU_01-01-2020_A_31-12-2020.CSV", delimiter=";", skiprows=8, encoding="ISO-8859-1")
df.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2020/01/01,0000 UTC,0,938,"938,1","937,5",NaN,"24,1","19,9",26,"23,9","19,9","17,5",78,60,77,169,"7,1",1,NaN
1,2020/01/01,0100 UTC,0,"938,7","938,7","937,9",NaN,"24,6","19,3","25,3","24,1","19,9",18,77,64,73,126,"5,3",",4",NaN
2,2020/01/01,0200 UTC,0,"938,4","938,7","938,4",NaN,"23,3",19,"24,8","23,3","19,4","18,4",77,68,77,107,"2,7",0,NaN
3,2020/01/01,0300 UTC,0,938,"938,4",938,NaN,"22,4","19,1","23,3","22,4","19,1","18,6",81,77,81,330,",9",0,NaN
4,2020/01/01,0400 UTC,0,"937,9",938,"937,9",NaN,"23,1",19,"23,2","22,1","19,2","18,6",82,75,77,308,"1,4",0,NaN


In [6]:
path = '../raw_data/SP'
files = [f for f in listdir(path) if isfile(join(path, f))]
print(files[0])
df = pd.read_csv(f'../raw_data/SP/{files[0]}', sep=';', skiprows=8, encoding="ISO-8859-1")
df.head()
files[2].split('_')
teste=df
teste['estacao']=files[0].split('_')[4]
teste.head()


INMET_SE_SP_A771_SAO PAULO - INTERLAGOS_01-01-2020_A_31-12-2020.CSV


,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,estacao
0,2020/01/01,0000 UTC,0,"923,1","923,1","922,1",NaN,22,"20,5","22,2",...,"20,5","20,3",92.0,90.0,92.0,172.0,"2,8",",4",NaN,SAO PAULO - INTERLAGOS
1,2020/01/01,0100 UTC,0,"923,7","923,7","923,1",NaN,"22,1","20,7","22,3",...,"20,8","20,4",92.0,90.0,92.0,101.0,"1,8",",9",NaN,SAO PAULO - INTERLAGOS
2,2020/01/01,0200 UTC,0,"923,6","923,8","923,6",NaN,"23,2","20,1","23,5",...,"20,8","20,1",92.0,82.0,82.0,7.0,"3,3","1,2",NaN,SAO PAULO - INTERLAGOS
3,2020/01/01,0300 UTC,0,"923,1","923,6","923,1",NaN,"23,3","19,5","23,6",...,"20,1","19,4",83.0,78.0,79.0,6.0,3,"1,2",NaN,SAO PAULO - INTERLAGOS
4,2020/01/01,0400 UTC,0,"922,6","923,1","922,6",NaN,"22,9",19,"23,6",...,"19,5",19,79.0,76.0,79.0,345.0,"3,5",",6",NaN,SAO PAULO - INTERLAGOS


In [31]:
#lista de colunas
col_list = []
for col in teste.columns:
    col_list.append(col)
col_list

#fazer list com os dataframes

df_list = []
for file in files:
    df = pd.read_csv(f'../raw_data/SP/{files[0]}', sep=';', skiprows=8, encoding="ISO-8859-1")
    df_list.append(df)
df_list
    
#fundir os dataframes no dataframe vazio

full_df = pd.concat(df_list)
full_df
df2 = full_df.copy()

#feature engineering

header=pd.read_csv(f'../raw_data/SP/{files[0]}', sep=';', nrows=7, encoding="ISO-8859-1")
df2['Estaçao']=header['SE'][1] 
df2['Latitude']=header['SE'][3] 
df2['Longitude']=header['SE'][4] 
df2['Altitude']=header['SE'][5] 


In [40]:



lat_log_alt = pd.read_csv(f'../raw_data/SP/{files[0]}', sep=';', skiprows=4, 
                          nrows=3, encoding="ISO-8859-1", decimal=',', names=['lat_lon_alt','valor'])


print(lat_log_alt['valor'][0])

#listas vazias para preencher com os dados
Estação_lista = []
Latitude_lista = []
Longitude_lista = []
Altitude_lista = []

#tamanho do dataframe
rows = df2.shape[0]

#loop para preencher as listas vazias
for row in range(rows):
    lat_log_alt = pd.read_csv(f'../raw_data/SP/{files[row]}', sep=';', skiprows=4, 
                          nrows=3, encoding="ISO-8859-1", decimal=',', names=['lat_lon_alt','valor'])
    Estação_lista.append(files[row].split('_')[4])
    Latitude_lista.append(lat_log_alt['valor'][0]) 
    Longitude_lista.append(lat_log_alt['valor'][1]) 
    Altitude_lista.append(lat_log_alt['valor'][2])

df2['Estação'] = Estação_lista
df2['Longitude'] = Longitude_lista
df2['Latitude'] = Latitude_lista
df2['Altitude'] = Altitude_lista

df2

#criar colunas com as listas preenchidas



-23.724501


IndexError: list index out of range

In [46]:
for teste in range(2):
    print(teste)

0
1


In [44]:
lat_log_alt = pd.read_csv(f'../raw_data/SP/{files[0]}', sep=';', skiprows=4, 
                      nrows=3, encoding="ISO-8859-1", decimal=',', names=['lat_lon_alt','valor'])

lat_log_alt

,lat_lon_alt,valor
0,LATITUDE:,-23.724501
1,LONGITUDE:,-46.677501
2,ALTITUDE:,771.000000
